<a href="https://colab.research.google.com/github/vikxoxo/NLP/blob/master/NER_using_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook 1: Pre-processing text data

1.       The CSV file from the ECB contains the entire text of a speech in a single line. We would like you to transform this into a dataframe in a slightly different format

2.       The resulting dataframe should have each sentence of the speech as a separate row. The dataframe should have the following columns – Date, Speaker, Title, Type of speech, Role of speaker, Occasion, SentenceText, SentenceID (SentenceID should be an index within the speech which you create so that the first sentence of the speech has SentenceID=0 and this then increments throughout the speech)

3.       Save the final dataframe as a pickle file

In [ ]:
"""
while working on the task I saved the dataset (.csv)
on my drive for quick access
"""
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import and Lib

Required dependencies

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 110.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.1 MB/s eta 0:00:00


In [ ]:
!pip install deep_translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 8.8 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=54f73f8052d50e72a79a089f1303aaf16a8a970772f3c7d706b829f1f5fbef02
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [ ]:
import os
import json
from deep_translator import GoogleTranslator
translator = GoogleTranslator(source='auto', target='en')

In [ ]:
"""
other tokenizers might also be tried out
but I found nltk is good enough for this use case.
"""
from nltk.tokenize import sent_tokenize

In [ ]:
import datetime

In [ ]:
from langdetect import detect

In [ ]:
import torch

In [ ]:
torch.device('cuda')

device(type='cuda')

## Functions-1

In [ ]:
def get_default_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
        # return 1
    else:
        return torch.device('cpu')
        # return -1

In [ ]:
def convert_2eng(text):
  """
  transalate input text to eng-text
  using Google api, handles the transalator 
  min length issue by tokenizing input in sent
  """
  lang = detect(str(text))
  if lang != 'en':
    if len(text)< 4990:
      en_text = translator.translate(text)
    # print(lang)
    else:
      en_text = ''
      sent_text = sent_tokenize(text)
      for sent in sent_text:
        eng_sent = translator.translate(sent)
        en_text = en_text + eng_sent
         
  else:
    en_text = text

  return en_text

## Read and Dev Code

In [ ]:
"""
read raw ecb speech csv
"""
folder_path = '/content/drive/MyDrive/hsbc_assignment/'
rawfile_name = 'all_ECB_speeches.csv'
complete_path = folder_path + rawfile_name
"""
for quick access I saved the dataset on drive
"""
# df_raw = pd.read_csv(r"/content/drive/MyDrive/hsbc_assignment/all_ECB_speeches.csv",
#                      sep='|'
#                     #  quoting=3,
#                     #  engine = 'python'
#                      )

"""
But data can be loaded from below link directly
"""
link2csv = 'https://www.ecb.europa.eu/press/key/shared/data/all_ECB_speeches.csv?ffe907b66d1cff51095884fe18833e9f'
df_raw = pd.read_csv(link2csv, sep='|')

In [ ]:
#random 10 sample
print(df_raw.sample(10))

            date                  speakers  \
2214  2003-10-29      Willem F. Duisenberg   
902   2014-11-21              Mario Draghi   
2167  2004-05-11  Gertrude Tumpel-Gugerell   
1394  2010-11-16              Jürgen Stark   
1424  2010-09-27  Gertrude Tumpel-Gugerell   
186   2021-03-01           Luis de Guindos   
2435  2000-09-18         Sirkka Hämäläinen   
105   2021-11-26         Christine Lagarde   
2213  2003-11-03  Gertrude Tumpel-Gugerell   
1992  2006-02-27       Jean-Claude Trichet   

                                                  title  \
2214  Address delivered on the occasion of a special...   
902                    Monetary policy in the euro area   
2167  Speech at the ECB-CFS Symposium 'The Role of t...   
1394                                     The new normal   
1424  The interplay of banking, financial intermedia...   
186   Macroprudential policy after the COVID-19 pand...   
2435  The euro and the dollar - new imperatives for ...   
105                  

In [ ]:
# no of speeches ie row in the dataset
print(len(df_raw))

2652


### check if more than one speaker is there for a row

In [ ]:
# very limited speakers with only one instance where more than one speaker is there for a row
# few rows with no speakers also there
df_raw['speakers'].unique()

array(['Christine Lagarde', 'Fabio Panetta', 'Philip R. Lane',
       'Frank Elderson', 'Isabel Schnabel', 'Luis de Guindos', nan,
       'Yves Mersch', 'Benoît Cœuré', 'Sabine Lautenschläger',
       'Mario Draghi', 'Peter Praet', 'Vítor Constâncio', 'Jörg Asmussen',
       'José Manuel González-Páramo', 'Lorenzo Bini Smaghi',
       'Jürgen Stark', 'Jean-Claude Trichet', 'Gertrude Tumpel-Gugerell',
       'Lucas Papademos', 'Otmar Issing', 'Tommaso Padoa-Schioppa',
       'Eugenio Domingo Solans', 'Willem F. Duisenberg',
       'Sirkka Hämäläinen', 'Christian Noyer',
       'Willem F. Duisenberg,Eugenio Domingo Solans',
       'Alexandre Lamfalussy'], dtype=object)

In [ ]:
#let's see the rows where more than one speakers or none speaker 
df_raw[df_raw['speakers']== 'Willem F. Duisenberg,Eugenio Domingo Solans']

,date,speakers,title,subtitle,contents
2368,2001-08-30,"Willem F. Duisenberg,Eugenio Domingo Solans",ECB Press conference on the occasion of the un...,Transcript of the questions asked and the answ...,ECB Press conference on the occasion of the ...


In [ ]:
df_raw[df_raw['speakers'].isnull()]

,date,speakers,title,subtitle,contents
42,2022-07-02,NaN,Isabel Schnabel: Inflation in the euro area - ...,"Presentation by Isabel Schnabel, Member of the...",NaN
156,2021-06-22,NaN,Die Geldpolitik der EZB in der Corona-Krise,"Vortrag von Isabel Schnabel, Mitglied des Dire...",NaN
179,2021-03-18,NaN,Die Geldpolitik der EZB in der Corona-Krise,"Vortrag von Isabel Schnabel, Mitglied des Dire...",NaN


### How many different langs

In [ ]:
from langdetect import detect
df_raw['lang'] = df_raw.apply(lambda x: detect(str(x['subtitle'])), axis=1)
# lang = detect(subtitle)

In [ ]:
df_raw['lang'].unique()

array(['en', 'de', 'tl', 'es', 'fr', 'it', 'lt', 'ca', 'nl'], dtype=object)

In [ ]:
df_raw['lang'].value_counts()

en    2461
de     115
es      40
fr      25
it       7
tl       1
lt       1
ca       1
nl       1
Name: lang, dtype: int64

It becomes important to check if hard-coded methods to retrieve occasion, speakers etc. work for each of the lang after eng conversion or not?

Ans: After Analysis it was found that hard coded methods do not work well on all instances, but works well for many eng speeches.

In [ ]:
#getting indexes of a particular lang
"""
few indexes to check: 
de: 32, 147, 1503, 1185
es: 383, 1228, 2619
fr: 520, 2611
it: 686, 2489
nl: 2508
tl: 207
lt: 2084
ca: 2248
"""
list(df_raw[df_raw['lang']=='ca'].index.values)

[2248]

### inspecting raw contents

It is seen that in this dataset, the reuired additional entities e.g. type of speech etc. can be retieved from subtitle using some hard-coded methods too for many of the eng speech instances, not for all. 

Commonly Followed Pattern for many eng instances:
{Type of Speech} by {Speaker Name}, {Occasion Name} {Event Name}, {Place}, {Date}

In [ ]:
row = 1111

In [ ]:
df_raw.loc[row,'speakers']

'Jörg Asmussen'

In [ ]:
df_raw.loc[row,'title']

'Is Europe on the right track?'

In [ ]:
df_raw.loc[row,'subtitle']

'Speech by Jörg Asmussen, Member of the Executive Board of the ECB, at Deutsche Bank Women in European Business conference “Competition versus Coalition - the quest for growth?”, Frankfurt am Main, 20 March 2013'

In [ ]:
subtitle = df_raw.loc[row,'subtitle']

In [ ]:
subtitle

'Speech by Jörg Asmussen, Member of the Executive Board of the ECB, at Deutsche Bank Women in European Business conference “Competition versus Coalition - the quest for growth?”, Frankfurt am Main, 20 March 2013'

In [ ]:
#check if the row language is en or not
#if not en convert in eng to get consistent results of type, occasion 
#to analyze further
from langdetect import detect
lang = detect(subtitle)
print(lang)
if lang != 'en':
  df_raw.loc[row,'en_subtitle']= translator.translate(subtitle)
else:
  df_raw.loc[row,'en_subtitle'] = df_raw.loc[row,'subtitle']



en


In [ ]:
df_raw.loc[row,'en_subtitle']

'Speech by Jörg Asmussen, Member of the Executive Board of the ECB, at Deutsche Bank Women in European Business conference “Competition versus Coalition - the quest for growth?”, Frankfurt am Main, 20 March 2013'

### Hard-Coded Methods

Note: Not utilized as it fails for non-eng lang, and not a reliable soution at all.

In [ ]:
#type of the speech (hard-coded: depends on dataset format, and works for speakers less than two)
df_raw.loc[row,'subtitle'].split(' by')[0]

'Speech'

In [ ]:
#type of the speech (hard-coded: depends on dataset format, and works for speakers less than two)
df_raw.loc[row,'en_subtitle'].split(' by')[0]

'Speech'

In [ ]:
#  role of the speaker (hard-coded: depends on dataset format, and works for speakers less than two)
df_raw.loc[row,'en_subtitle'].split(',')[1]

' Member of the Executive Board of the ECB'

In [ ]:
# complete role of the speaker (hard-coded: depends on dataset format, and works for speakers less than two)
df_raw.loc[row,'en_subtitle'].split(',')[1]+ ',' + df_raw.loc[row,'subtitle'].split(',')[2]

' Member of the Executive Board of the ECB, at Deutsche Bank Women in European Business conference “Competition versus Coalition - the quest for growth?”'

In [ ]:
# Occasion (hard-coded: depends on dataset format, and works for speakers less than two)
df_raw.loc[row,'en_subtitle'].split(',')[2]

' at Deutsche Bank Women in European Business conference “Competition versus Coalition - the quest for growth?”'

In [ ]:
df_raw.loc[row,'contents']

'  Is Europe on the right track?   Speech by Jörg Asmussen, Member of the Executive Board of the ECB, at Deutsche Bank Women in European Business conference“Competition versus Coalition – the quest for growth?”, Frankfurt am Main, 20 March 2013 Dear Ms. Siebert, Dear Mr. Leithner, Ladies and gentlemen, Thank you very much for inviting me to speak at the Women in European Business Conference organised by Deutsche Bank. The topic of this year’s conference – the quest for growth – could scarcely be more relevant for Europe today. As this audience knows better than most, the low growth environment is posing serious challenges for businesses. Uncertainty about the future is a drag on confidence and a barrier to new investment. It causes banks to cut back on lending, especially to small- and medium-sized firms, which are critical to help growth rebound. And above all, low growth is deeply affecting the lives of ordinary people in Europe. As a result, many are frustrated, and some are even de

## Can we use QA system to get speakers, occasion etc

Yes! very well working :)

Different QA transfomers models were tried out with different question prompts.

The ones which found to be working well are utilized for entity extraction over whole dataset

In [ ]:
"""
As we saw that the dataset is very well curated, and there
is only few instance where the speaker information is not conistent to get
required fields (speakers, thier roles, occasion etc.) using
dataset specific split methods

But Can we use pretrained transformers to get these fields let's try!
But compute and memory expenses are just a minute drawback
"""

"\nAs we saw that the dataset is very well curated, and there\nis only few instance where the speaker information is not conistent to get\nrequired fields (speakers, thier roles, occasion etc.) using\ndataset specific split methods\n\nBut Can we use pretrained transformers to get these fields let's try!\nBut compute and memory expenses\n"

In [ ]:
device = get_default_device()

In [ ]:
from transformers import pipeline

# model_name = 'distilbert-base-cased-distilled-squad' #not works
# model_name = "bert-large-uncased-whole-word-masking-finetuned-squad" #not works
# model_name = 'deepset/roberta-large-squad2' #not works
model_name = 'ahotrod/electra_large_discriminator_squad2_512'
question_answerer = pipeline("question-answering", model= model_name,
                             device=0 if torch.cuda.is_available() else -1)

subtitle = df_raw.loc[row,'en_subtitle']




In [ ]:
subtitle

'Speech by Jörg Asmussen, Member of the Executive Board of the ECB, at Deutsche Bank Women in European Business conference “Competition versus Coalition - the quest for growth?”, Frankfurt am Main, 20 March 2013'

In [ ]:
context = subtitle

result = question_answerer(question="Who were the speakers?", context=context)
print(
f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'Deutsche Bank Women in European Business conference', score: 0.1408, start: 70, end: 121


In [ ]:
#type of speech
result = question_answerer(question="What was delivered by speaker?", context=context)
print(
f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: '“Competition versus Coalition - the quest for growth?', score: 0.2164, start: 122, end: 175


In [ ]:
#type of speech (generally better)
result = question_answerer(question="What was the type of text?", context=context)
print(
f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'Speech', score: 0.4129, start: 0, end: 6


In [ ]:
#occasion name
result = question_answerer(question="What was the occasion?", context=context)
occasion = result['answer']
print(
f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'Deutsche Bank Women in European Business conference', score: 0.7865, start: 70, end: 121


In [ ]:
#occasion name (generally better)
result = question_answerer(question="What was the name of the event?", context=context)
occasion = result['answer']
print(
f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'Deutsche Bank Women in European Business conference', score: 0.2028, start: 70, end: 121


In [ ]:
#occasion place
result = question_answerer(question="What was the place of the event?", context=context)
print(
f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'Frankfurt am Main', score: 0.9749, start: 178, end: 195


In [ ]:
#occasion place (generally better)
q = "Where did " + occasion + "happen ?"
result = question_answerer(question=q, context=context)
print(
f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'Frankfurt am Main', score: 0.9747, start: 178, end: 195


In [ ]:
#role of the speaker
speakers_ = df_raw.loc[row,'speakers'].split(',')
print(speakers_)
for speaker in speakers_:
  print(speaker)
  q = "Who is " + speaker + " ?"
  result = question_answerer(question=q, context=context)
  print(
  f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

['Jörg Asmussen']
Jörg Asmussen
Answer: 'Member of the Executive Board of the ECB', score: 0.9842, start: 25, end: 65


In [ ]:
#role of the speaker
speakers_ = df_raw.loc[row,'speakers'].split(',')
print(speakers_)
for speaker in speakers_:
  print(speaker)
  q = "What is the role of the" + speaker + " ?"
  result = question_answerer(question=q, context=context)
  print(
  f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

['Jörg Asmussen']
Jörg Asmussen
Answer: 'Member of the Executive Board of the ECB', score: 0.9346, start: 25, end: 65


## Can we use NER for getting answers?

NER is another common solution for such entity extraction but it is not that reliable as:

*   More than one entity might be present from the same class, e.g. Country as a Place, and event city as a place
*   Wrong type: role of the speaker can be recognized as ORG
*  Precision is lost 
* Lots of Preprocessing required



In [ ]:
subtitle = df_raw.loc[row,'en_subtitle']

In [ ]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

In [ ]:
raw_text = subtitle
text1= NER(raw_text)

In [ ]:
for word in text1.ents:
    print(word.text,word.label_)

Speech ORG
Jörg Asmussen PERSON
the Executive Board ORG
ECB ORG
Deutsche Bank Women ORG
European NORP
Competition versus Coalition - the WORK_OF_ART
Frankfurt GPE
Main PERSON
20 March 2013 DATE


## Add Required Entity Columns using QA models

Date, Speaker, Title, Type of speech, Role of speaker, Occasion

Different Functions are created, and two columns are created for each entity.

e.g. 'Type of Speech' , 'Type of Speech - 2' using two different Q prompts

In [ ]:
def get_speech_type(context):
  #type of speech
  result2 = question_answerer(question="What was delivered by speaker?", context=context)

  #type of speech (generally better)
  result1 = question_answerer(question="What was the type of text?", context=context)

  return result1['answer'], result2['answer']

In [ ]:
def get_speaker_role(context, speakers):
  #role of the speaker
  context = str(context)
  speakers = str(speakers)
  speakers_ = speakers.split(',')
  # print(speakers_)
  results = []
  for speaker in speakers_:
    # print(speaker)
    q = "Who is " + speaker + " ?"
    result = question_answerer(question=q, context=context)
    results.append(result['answer'])

  result1 = ' '.join(results)

  results = []
  for speaker in speakers_:
    # print(speaker)
    q = "What is the role of the" + speaker + " ?"
    result = question_answerer(question=q, context=context)
    results.append(result['answer'])

  result2 = ' '.join(results)

  return result1, result2

In [ ]:
def get_occasion(context):
  context = str(context)
  #occasion name (generally better)
  result = question_answerer(question="What was the name of the event?", context=context)
  result1 = result['answer']

  #occasion name
  result = question_answerer(question="What was the occasion?", context=context)
  result2 = result['answer']

  return result1, result2

In [ ]:
def get_occasion_place(occasion, context):
  #occasion place (generally better)
  q = "Where did " + str(occasion) + "happen ?"
  context = str(context)
  result1 = question_answerer(question=q, context=context)

  #occasion place
  result2 = question_answerer(question="What was the place of the event?", context=context)

  return result1['answer'], result2['answer']


In [ ]:
df_ans_nosent = df_raw.copy(deep=True)

In [ ]:
list(df_ans_nosent.columns)

['date', 'speakers', 'title', 'subtitle', 'contents', 'lang', 'en_subtitle']

In [ ]:
# convert the sub-title and titlein english
df_ans_nosent['en_title'] = df_ans_nosent.apply(lambda x: convert_2eng(x['title']), axis=1)


In [ ]:
df_ans_nosent['en_subtitle'] = df_ans_nosent.apply(lambda x: convert_2eng(str(x['subtitle'])), axis=1)
# df_ans_nosent['en_contents'] = df_ans_nosent.apply(lambda x: convert_2eng(str(x['contents'])), axis=1)

#detecting the lang 
df_ans_nosent['lang'] = df_ans_nosent.apply(lambda x: detect(str(x['subtitle'])), axis=1)

In [ ]:
df_ans_nosent['en_subtitle'].sample(10)

1227    Speech by Mario Draghi, President of the ECB a...
2166    by Professor Otmar Issing, Member of the Execu...
1442    Intervention by Gertrude Tumpel-Gugerell, Memb...
1178    Speech by Mario Draghi, President of the ECB, ...
342     Intervention by Luis de Guindos, Vice-Presiden...
2078    dr Gertrude Tumpel-Gugerell Member of the Gove...
1966    Speech by Jean-Claude Trichet, President of th...
214     Keynote speech by Luis de Guindos, Vice-Presid...
1975    Speech by Otmar Issing, Member of the Executiv...
402     Speech by Mario Draghi, President of the Europ...
Name: en_subtitle, dtype: object

In [ ]:
#speech type
df_ans_nosent[['Type of speech','Type of speech 2']] = df_ans_nosent.apply(lambda x: get_speech_type(x['en_subtitle']), axis=1, result_type ='expand')

In [ ]:
#role of speaker
df_ans_nosent[['Role of speaker','Role of speaker 2']] = df_ans_nosent.apply(lambda x: get_speaker_role(x['en_subtitle'], x['speakers']), axis=1, result_type ='expand')

In [ ]:
#occasion (e.g. a conference)
df_ans_nosent[['Occasion','Occasion 2']] = df_ans_nosent.apply(lambda x: get_occasion(x['en_subtitle']), axis=1, result_type ='expand')

In [ ]:
#occasion place (e.g. city or country place)
df_ans_nosent[['Occasion Place', 'Occasion Place 2']] = df_ans_nosent.apply(lambda x: get_occasion_place(x['Occasion'],x['en_subtitle']), axis=1, result_type ='expand')

In [ ]:
print(df_ans_nosent.sample(5))

            date                     speakers  \
49    2022-06-15                Fabio Panetta   
1243  2012-02-19                 Benoît Cœuré   
2416  2000-11-17         Willem F. Duisenberg   
2114  2004-11-10                 Otmar Issing   
1342  2011-04-14  José Manuel González-Páramo   

                                                  title  \
49    The digital euro and the evolution of the fina...   
1243  Central banks and the challenges of the zero l...   
2416                 The international role of the euro   
2114         Der Euro - Eine stabile Währung für Europa   
1342  Risk, return, resilience: The future financial...   

                                               subtitle  \
49    Introductory statement by Fabio Panetta, Membe...   
1243  Intervention by Benoît Cœuré, Member of the Ex...   
2416  Keynote address by Dr. Willem F. Duisenberg, P...   
2114  Rede von Otmar Issing, Mitglied des Direktoriu...   
1342  Speech by José Manuel González-Páramo, Member ...

In [ ]:
"""
uncomment to save this intermediate csv
this csv does not split speech contets into sent,
yet without sentence split it can be utilized for task-2 analytics, and visualizations
"""
# file_name = 'task1_nosent.csv'
# df_ans_nosent.to_csv(folder_path+file_name)

## Sent Tokenizer

https://www.analyticsvidhya.com/blog/2019/07/how-get-started-nlp-6-unique-ways-perform-tokenization/

In [ ]:
# one of the speech
text = df_raw.loc[row,'contents']

In [ ]:
# #did not work out
# import dateutil.parser as dparser
# """
# can be utilized for first date detection
# in content, and content after date can be considered for the
# sent token process
# """
# # extracting date using inbuilt func.
# res = dparser.parse(text, fuzzy=True)
  
# # printing result
# print("Computed date : " + str(res))

https://stackabuse.com/how-to-format-dates-in-python/

In [ ]:
i_date = df_raw.loc[row,'date'].split('-')
# format
in_format = '%Y-%m-%d'
out_format = '%d %B %Y'
x = datetime.datetime(int(i_date[0]), int(i_date[1]), int(i_date[2]))
print(x)
print(x.strftime(out_format))
# text = df_raw.loc[row,'contents'].split(date)[1]

2013-03-20 00:00:00
20 March 2013


each of the speech has a common format in this dataset.

common format/pattern:

{sub-title part} {Date} {Actual Speech Transcript}

In [ ]:
text

'  Is Europe on the right track?   Speech by Jörg Asmussen, Member of the Executive Board of the ECB, at Deutsche Bank Women in European Business conference“Competition versus Coalition – the quest for growth?”, Frankfurt am Main, 20 March 2013 Dear Ms. Siebert, Dear Mr. Leithner, Ladies and gentlemen, Thank you very much for inviting me to speak at the Women in European Business Conference organised by Deutsche Bank. The topic of this year’s conference – the quest for growth – could scarcely be more relevant for Europe today. As this audience knows better than most, the low growth environment is posing serious challenges for businesses. Uncertainty about the future is a drag on confidence and a barrier to new investment. It causes banks to cut back on lending, especially to small- and medium-sized firms, which are critical to help growth rebound. And above all, low growth is deeply affecting the lives of ordinary people in Europe. As a result, many are frustrated, and some are even de

In [ ]:
date_to_split = x.strftime(out_format)


In [ ]:
speech_text_list = text.split(date_to_split)

In [ ]:
print(len(speech_text_list))

2


In [ ]:
sppech_text = ' '.join(speech_text_list[1:])

In [ ]:
#{Actual Speech Portion}
speech_text

' Dear Ms. Siebert, Dear Mr. Leithner, Ladies and gentlemen, Thank you very much for inviting me to speak at the Women in European Business Conference organised by Deutsche Bank. The topic of this year’s conference – the quest for growth – could scarcely be more relevant for Europe today. As this audience knows better than most, the low growth environment is posing serious challenges for businesses. Uncertainty about the future is a drag on confidence and a barrier to new investment. It causes banks to cut back on lending, especially to small- and medium-sized firms, which are critical to help growth rebound. And above all, low growth is deeply affecting the lives of ordinary people in Europe. As a result, many are frustrated, and some are even desperate. They doubt whether the European strategy to deal with the crisis is the right one. They are asking: “Will we ever come out of it, with ever more cuts and austerity?”  And it is right and legitimate that they are asking this. “Power of

In [ ]:
#sent tokenization
sent_tokenize(speech_text)

[' Dear Ms. Siebert, Dear Mr. Leithner, Ladies and gentlemen, Thank you very much for inviting me to speak at the Women in European Business Conference organised by Deutsche Bank.',
 'The topic of this year’s conference – the quest for growth – could scarcely be more relevant for Europe today.',
 'As this audience knows better than most, the low growth environment is posing serious challenges for businesses.',
 'Uncertainty about the future is a drag on confidence and a barrier to new investment.',
 'It causes banks to cut back on lending, especially to small- and medium-sized firms, which are critical to help growth rebound.',
 'And above all, low growth is deeply affecting the lives of ordinary people in Europe.',
 'As a result, many are frustrated, and some are even desperate.',
 'They doubt whether the European strategy to deal with the crisis is the right one.',
 'They are asking: “Will we ever come out of it, with ever more cuts and austerity?”  And it is right and legitimate tha

## Applying funct to add sentence rows with required columns

https://developers.google.com/admin-sdk/directory/v1/languages

https://stackoverflow.com/questions/70930127/nltk-available-languages-for-word-tokenization-and-sentence-tokenization

The dataframe should have the following columns – Date, Speaker, Title, Type of speech, Role of speaker, Occasion, SentenceText, SentenceID (SentenceID should be an index within the speech which you create so that the first sentence of the speech has SentenceID=0 and this then increments throughout the speech)

In [ ]:
def lang_google_2nltk(lang):
  lang_dict = {'de': 'german', 
               'es': 'spanish',
               'fr': 'french',
               'it': 'italian',
               'nl': 'dutch' ,
               'tl': 'english', #filipino seems an error
               'lt': 'polish', #lithuanian closest to polish
               'ca': 'spanish', #catalan closest to spanish
               'en': 'english'
               }
  return lang_dict[lang]

In [ ]:
def get_sentences(contents, date, lang):
  i_date = date.split('-')
  # format
  in_format = '%Y-%m-%d'
  out_format = '%d %B %Y'
  x = datetime.datetime(int(i_date[0]), int(i_date[1]), int(i_date[2]))
  # print(x)
  # print(x.strftime(out_format))
  # text = df_raw.loc[row,'contents'].split(date)[1]
  date_to_split = x.strftime(out_format)
  contents = str(contents)
  speech_text_list = contents.split(date_to_split)
  if len(speech_text_list)>=2:
    speech_text = ' '.join(speech_text_list[1:])
  else:
    speech_text = contents
  
  lang_ = lang_google_2nltk(lang)
  
  return sent_tokenize(speech_text, language= lang_)

In [ ]:
#reading the intermediate created csv having rest of the required columns
# folder_path = '/content/drive/MyDrive/'
# file_name = 'task1_nosent.csv'
# df_ans_nosent = pd.read_csv(folder_path+file_name)

In [ ]:
df_ans_nosent.columns

Index(['Unnamed: 0', 'date', 'speakers', 'title', 'subtitle', 'contents',
       'lang', 'en_subtitle', 'en_title', 'Type of speech', 'Type of speech 2',
       'Role of speaker', 'Role of speaker 2', 'Occasion', 'Occasion 2',
       'Occasion Place', 'Occasion Place 2'],
      dtype='object')

In [ ]:
df_ans_nosent.loc[0, :].to_dict()

{'Unnamed: 0': 0,
 'date': '2022-12-08',
 'speakers': 'Christine Lagarde',
 'title': 'Macroprudential policy in Europe: building resilience in a challenging environment',
 'subtitle': 'Welcome remarks by Christine Lagarde, President of the ECB and Chair of the European Systemic Risk Board, at the sixth annual conference of the ESRB',
 'contents': '  SPEECH  Macroprudential policy in Europe: building resilience in a challenging environment   Welcome remarks by Christine Lagarde, President of the ECB and Chair of the European Systemic Risk Board, at the sixth annual conference of the ESRB Frankfurt am Main, 8 December 2022 It is my pleasure to welcome you to the sixth annual conference of the European Systemic Risk Board (ESRB).[1]  When future historians look back on our times, they may well say we lived through an era of “permacrisis”. An array of powerful shocks –  the pandemic, Russia’s unjustifiable invasion of Ukraine, and the energy crisis – have hit the global economy in quick su

In [ ]:
#date, speaker, title already there
"""
Intsead of using whole dataset first 110 speeches are used here
One may obviously use whole dataset just by expanding the 
limit variable in this cell
"""
df_ans_sent_rows = []
r = 0
limit = 110
for row in range(len(df_ans_nosent)):
  if row> limit:
    break
  #getting ans_nosent row as dict
  df2 = df_ans_nosent.loc[row, :].to_dict()
  contents1 = df2['contents']
  date1 = df2['date']
  lang = df2['lang']
  sent_list = get_sentences(contents1, date1, lang)
  i = 0
  for sent in sent_list:
    print(r)
    sent_dic = df2.copy()
    # deleting contents
    del sent_dic['contents']
    sent_dic['SentenceText'] = sent
    if lang != 'eng':
      sent_dic['en_SentenceText'] = translator.translate(sent)
    else:
      sent_dic['en_SentenceText'] = sent
    sent_dic['SentenceID'] = i
    df_ans_sent_rows.append(sent_dic)
    i = i+1
    r = r+1

 

  



Streaming output truncated to the last 5000 lines.
7506
7507
7508
7509
7510
7511
7512
7513
7514
7515
7516
7517
7518
7519
7520
7521
7522
7523
7524
7525
7526
7527
7528
7529
7530
7531
7532
7533
7534
7535
7536
7537
7538
7539
7540
7541
7542
7543
7544
7545
7546
7547
7548
7549
7550
7551
7552
7553
7554
7555
7556
7557
7558
7559
7560
7561
7562
7563
7564
7565
7566
7567
7568
7569
7570
7571
7572
7573
7574
7575
7576
7577
7578
7579
7580
7581
7582
7583
7584
7585
7586
7587
7588
7589
7590
7591
7592
7593
7594
7595
7596
7597
7598
7599
7600
7601
7602
7603
7604
7605
7606
7607
7608
7609
7610
7611
7612
7613
7614
7615
7616
7617
7618
7619
7620
7621
7622
7623
7624
7625
7626
7627
7628
7629
7630
7631
7632
7633
7634
7635
7636
7637
7638
7639
7640
7641
7642
7643
7644
7645
7646
7647
7648
7649
7650
7651
7652
7653
7654
7655
7656
7657
7658
7659
7660
7661
7662
7663
7664
7665
7666
7667
7668
7669
7670
7671
7672
7673
7674
7675
7676
7677
7678
7679
7680
7681
7682
7683
7684
7685
7686
7687
7688
7689
7690
7691
7692
7693
7694
7695

KeyboardInterrupt: ignored

In [ ]:
df_ans_sent = pd.DataFrame(df_ans_sent_rows)  

In [ ]:
df_ans_sent.sample(5)

,Unnamed: 0,date,speakers,title,subtitle,lang,en_subtitle,en_title,Type of speech,Type of speech 2,Role of speaker,Role of speaker 2,Occasion,Occasion 2,Occasion Place,Occasion Place 2,SentenceText,en_SentenceText,SentenceID
8878,76,2022-03-24,Frank Elderson,Proportioning policy action to the evidence: m...,"Keynote speech by Frank Elderson, Member of th...",en,"Keynote speech by Frank Elderson, Member of th...",Proportioning policy action to the evidence: m...,webinar,Keynote speech,Member of the Executive Board of the ECB,Vice-Chair of the Supervisory Board of the ECB,The Institute of International & European Affa...,Institute of International & European Affairs ...,the ECB and Vice-Chair of the Supervisory Boar...,Institute of International & European Affairs,The economic analysis focuses on real and nomi...,The economic analysis focuses on real and nomi...,41
9025,77,2022-03-22,Fabio Panetta,Building a robust and diversified clearing eco...,"Welcome address by Fabio Panetta, Member of th...",en,"Welcome address by Fabio Panetta, Member of th...",Building a robust and diversified clearing eco...,Welcome address,Welcome address,Member of the Executive Board of the ECB,Member of the Executive Board of the ECB,Fourth Annual Joint Conference of the Deutsche...,Fourth Annual Joint Conference of the Deutsche...,Federal Reserve Bank of Chicago,Federal Reserve Bank of Chicago,"For example, in 2020 SIX acquired Bolsas y Mer...","For example, in 2020 SIX acquired Bolsas y Mer...",58
1419,11,2022-11-14,Luis de Guindos,Outlook for the euro area economy and financia...,"Speech by Luis de Guindos, Vice-President of t...",en,"Speech by Luis de Guindos, Vice-President of t...",Outlook for the euro area economy and financia...,Speech,25th Frankfurt Euro Finance Week,Vice-President of the ECB,Vice-President of the ECB,25th Frankfurt Euro Finance Week,25th Frankfurt Euro Finance Week,ECB,Frankfurt Euro Finance Week,"We will proceed with prudence, continuing to n...","We will proceed with prudence, continuing to n...",84
5251,49,2022-06-15,Fabio Panetta,The digital euro and the evolution of the fina...,"Introductory statement by Fabio Panetta, Membe...",en,"Introductory statement by Fabio Panetta, Membe...",The digital euro and the evolution of the fina...,Introductory statement,Introductory statement,Member of the Executive Board of the ECB,Member of the Executive Board of the ECB,Committee on Economic and Monetary Affairs of ...,Committee on Economic and Monetary Affairs of ...,ECB,the Committee on Economic and Monetary Affairs...,I now look forward to our discussion.,I now look forward to our discussion.,83
10083,88,2022-02-18,Fabio Panetta,Central bank digital currencies: defining the ...,"Contribution by Fabio Panetta, Member of the E...",en,"Contribution by Fabio Panetta, Member of the E...",Central bank digital currencies: defining the ...,digital,panel discussion on central bank digital curre...,Member of the Executive Board of the ECB,Member of the Executive Board of the ECB,US Monetary Policy Forum,US Monetary Policy Forum,ECB,US Monetary Policy Forum,But this would be tantamount to outsourcing th...,But this would be tantamount to outsourcing th...,26


In [ ]:
df_ans_sent.shape

(12505, 19)

In [ ]:
df_ans_sent['Unnamed: 0'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111])

In [ ]:
df_ans_sent['lang'].unique()

array(['en', 'de'], dtype=object)

In [ ]:
df_ans_sent.loc[9630]

Unnamed: 0                                                          84
date                                                        2022-02-28
speakers                                                 Fabio Panetta
title                Small steps in a dark room: guiding policy on ...
subtitle             Speech by Fabio Panetta, Member of the Executi...
lang                                                                en
en_subtitle          Speech by Fabio Panetta, Member of the Executi...
en_title             Small steps in a dark room: guiding policy on ...
Type of speech                                                  Speech
Type of speech 2                                        online seminar
Role of speaker               Member of the Executive Board of the ECB
Role of speaker 2             Member of the Executive Board of the ECB
Occasion                                             an online seminar
Occasion 2                                           an online seminar
Occasi

In [ ]:
"""
saving csv
"""
df_ans_sent.to_csv(folder_path+ 'task1_110speech.csv')

In [ ]:
#saving pickle
df_ans_sent.to_pickle(folder_path+ 'task1_110speech.pkl')

## Analytics

Can be done over the dataset for Task 2
Available in task-2 notebook

speech wise, speaker wise: n-gram, word-cloud analysis (Done)

Speech wise topic modelining (Done)

Speaker level topic distribution (Done)

Speech wise summary (Done)

Ocassion (Date) wise summary

Topic modeling visualization (Done)

Lang freq distribution

Toxic words analysis in speech

Finacial Sentiment: positive, negative, neutral
FinBert Sentiment Analysis